# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [10]:
pip install pandas

: 

In [ ]:
# imports
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import pandas as pd

: 

In [3]:
# constants

# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()


# OLLAMA_API = "http://localhost:11434/api/chat"
# HEADERS = {"Content-Type": "application/json"}
# MODEL = "gemma-2b"

# csv file path
file_path = 'sitemap.csv'  # Replace with your file path

API key looks good so far


In [5]:
# set up environment
def read_urls_from_csv(file_path, urls_column='URL'):
    """
    Reads URLs from a CSV file and returns them as a dictionary.

    :param file_path: Path to the CSV file.
    :param urls_column: The column name containing the URLs.
    :return: A dictionary with row indices as keys and URLs as values.
    """
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)

        # Create dictionary from specified column
        urls_dict = {index: row[urls_column] for index, row in df.iterrows()}

        return urls_dict

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
urls_dict = read_urls_from_csv(file_path, urls_column='URL')
print(urls_dict)

An error occurred: [Errno 2] No such file or directory: 'sitemap.csv'
None


In [6]:
# prompts

system_prompt = """You are a web application developer and security consultant. Your role is to alanyse URLS and detemine \
with a level of accuracy whick URLS might be similar in function.
Respond in markdown.\
Group similar urls into groups."""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

def get_url_user_prompt(url):
    user_prompt = ""  # Initialize the variable
    user_prompt += f"Here is the list of URLs to analyse.\n"
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [7]:
# here is the question; type over this to ask something new

def analyse_urls(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_url_user_prompt(url)}
        ],
    )
    result = response.choices[0].message['content']
    display(Markdown(result))

# question = """
# Please explain what this code does and why:
# yield from {book.get("author") for book in books if book.get("author")}
# """

In [ ]:
# Get gpt-4o-mini to answer, with streaming
# return(messages)



def stream_url(url):
    stream = openai.chat.completions.create(
        model=MODEL,
        url=urls_dict,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_url_user_prompt(url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

return stream_url(url)

TypeError: Completions.create() got an unexpected keyword argument 'url'

In [50]:
stream_url(url)

NameError: name 'url' is not defined

In [ ]:
# Get Llama 3.2 to answer